In [144]:
import os

In [145]:
from stop_words import get_stop_words

In [146]:
stop_words = get_stop_words('russian')

In [147]:
# stop_words

In [148]:
from pymystem3 import Mystem

In [149]:
mystem = Mystem()

In [150]:
def read_txt(path):
    for root, dirs, files in os.walk(path):
        for filename in files:
            open_name = os.path.join(root, filename)
            with open(open_name, 'r', encoding='utf-8') as f:
                text = f.read()
                yield text

In [151]:
for i in read_txt('./corpus/Otzyvy'):
    print(i)
    break

Отличный отель для бюджетного отдыха! приехали в самом начале мая, когда заполнен отель был далеко не на 100% и получили массу удовольствия! Очень чистая, ухоженная территория, замечательные, чистые номера (наш назывался фемили делюкс, был угловым, окна выходили на дорожку и забор, за которым, по-моему, ничего не находилось, поэтому неважная звукоизоляция нас совсем не беспокоила. Возможно, номера маловаты, но нам двоим нашего двухкомнатного хватило выше крыши). Персонал отеля - выше всяких похвал! Приветливые, старательные - на мой взгляд, просто безупречные. Где-то к 5-6 мая народу в отеле прибавилось, появилась "Жанна из Донецка" и компания, которая вспугнула своей активностью отдыхающих болгар, но на качестве работы персонала это никак не отразилось. В общем, настоящие молодцы! Несмотря на то, что отель практически на 100% "заточен" под нас, русских-украинцев, публика собралась вполне приличная, я, по крайней мере, "Тагила" там не заметила (Жанна из Донецка напоминала его весьма от

## Log Likelihood

In [152]:
from collections import defaultdict

freq_otzyvy = defaultdict(int)
freq_lenta = defaultdict(int)

for i in read_txt('./corpus/Otzyvy'):
    lemmas = mystem.lemmatize(i)
    for lemma in lemmas:
        if any([x.isalpha() for x in lemma]):
            if lemma not in stop_words:
                freq_otzyvy[lemma] += 1

for i in read_txt('./corpus/Lenta'):
    lemmas = mystem.lemmatize(i)
    for lemma in lemmas:
        if any([x.isalpha() for x in lemma]):
            if lemma not in stop_words:
                freq_lenta[lemma] += 1

In [153]:
sorted(freq_otzyvy.items(), key=lambda x: x[1], reverse=True)[:10]

[('отель', 129),
 ('номер', 33),
 ('пляж', 23),
 ('море', 22),
 ('бассейн', 20),
 ('отдых', 20),
 ('большой', 20),
 ('свой', 19),
 ('отдыхать', 17),
 ('вечер', 17)]

In [154]:
sorted(freq_lenta.items(), key=lambda x: x[1], reverse=True)[:10]

[('заявлять', 388),
 ('свой', 385),
 ('сообщать', 326),
 ('становиться', 253),
 ('дело', 246),
 ('президент', 239),
 ('слово', 239),
 ('власть', 202),
 ('россия', 184),
 ('ранее', 180)]

In [155]:
corpus_otzyvy_size = sum(freq_otzyvy.values())
corpus_lenta_size = sum(freq_lenta.values())

In [156]:
corpus_otzyvy_size, corpus_lenta_size

(3620, 64406)

Corpus1 - отзывы, Corpus2 - лента

In [157]:
import math

def LL(freq_corpus1, freq_corpus2, word):
    e1 = len(freq_corpus1) * (freq_corpus1[word] + freq_corpus2[word]) / (len(freq_corpus1) + len(freq_corpus2))
    e2 = len(freq_corpus2) * (freq_corpus1[word] + freq_corpus2[word]) / (len(freq_corpus1) + len(freq_corpus2))
    ll = 2 * (freq_corpus1[word] * math.log(freq_corpus1[word]/e1) + freq_corpus2[word] * math.log(freq_corpus2[word]/e2))
    return ll

In [158]:
ll_rank = defaultdict(int)

for word in freq_otzyvy.keys() & freq_lenta.keys():
    ll_rank[word] = LL(freq_otzyvy, freq_lenta, word)
    
sorted_ll_rank = [z[0] for z in sorted(ll_rank.items(), key=lambda x: x[1], reverse=True)]

## Top-10 and bottom-10

In [159]:
top10 = tuple(sorted_ll_rank[:10])

In [160]:
bottom10 = tuple(sorted_ll_rank[-10:])

In [170]:
## Я выбрала такие слова:

In [161]:
my_choice10 = ('отель', 'отдых', 'аэропорт', 'море', 'бассейн', 'вода', 'номер', 'ресторан', 'питание', 'отзыв')

In [162]:
words = top10 + bottom10 + my_choice10

In [163]:
len(set(top10) & set(my_choice10)) / 10

0.4

In [164]:
print('{:15s} | {:17s} | {:15s} | {:7s} | {:7s}'.format(
    "Слово", "Частота в отзывах", "Частота в ленте", "LL", "LL rank"
))
for i, word in enumerate(words):
    if i%10 == 0:
        print("="*73)
    print('{:15s} | {:17d} | {:15d} | {:7.3f} | {:7d}'.format(
        word, freq_otzyvy[word], freq_lenta[word], ll_rank[word], sorted_ll_rank.index(word) + 1
    ))

Слово           | Частота в отзывах | Частота в ленте | LL      | LL rank
отель           |               129 |               2 | 470.902 |       1
заявлять        |                 2 |             388 | 107.853 |       2
бассейн         |                20 |               1 |  68.399 |       3
отдых           |                20 |               1 |  68.399 |       4
становиться     |                 3 |             253 |  60.492 |       5
дело            |                 3 |             246 |  58.398 |       6
море            |                22 |               6 |  56.570 |       7
слово           |                 3 |             239 |  56.309 |       8
россия          |                 1 |             184 |  50.802 |       9
украина         |                 1 |             168 |  45.816 |      10
род             |                 1 |               6 |   0.002 |     981
направляться    |                 1 |               6 |   0.002 |     982
выставлять      |                 1 | 

## Chi-2

In [165]:
import pandas as pd
import numpy as np

In [166]:
from scipy.stats import chi2_contingency

In [167]:
def chi(freq_corpus1, freq_corpus2, word):
    obs = np.array([
        [freq_corpus1[word], freq_corpus2[word]],
        [sum(freq_corpus1.values()) - freq_corpus1[word], sum(freq_corpus2.values()) - freq_corpus2[word]]
        ])
    return chi2_contingency(obs)[0]

In [168]:
chi_rank = defaultdict(int)

for word in freq_otzyvy.keys() & freq_lenta.keys():
    chi_rank[word] = chi(freq_otzyvy, freq_lenta, word)
    
sorted_chi_rank = [z[0] for z in sorted(chi_rank.items(), key=lambda x: x[1], reverse=True)]

In [171]:
print('{:15s} | {:17s} | {:15s} | {:7s} | {:7s}'.format(
    "Слово", "Частота в отзывах", "Частота в ленте", "Chi", "Chi rank"
))
for i, word in enumerate(words):
    if i%10 == 0:
        print("="*73)
    print('{:15s} | {:17d} | {:15d} | {:7.3f} | {:7d}'.format(
        word, freq_otzyvy[word], freq_lenta[word], chi_rank[word], sorted_chi_rank.index(word) + 1
    ))

Слово           | Частота в отзывах | Частота в ленте | Chi     | Chi rank
отель           |               129 |               2 | 2242.020 |       1
заявлять        |                 2 |             388 |  17.055 |     128
бассейн         |                20 |               1 | 319.476 |       2
отдых           |                20 |               1 | 319.476 |       3
становиться     |                 3 |             253 |   7.975 |     210
дело            |                 3 |             246 |   7.606 |     214
море            |                22 |               6 | 283.942 |       4
слово           |                 3 |             239 |   7.239 |     217
россия          |                 1 |             184 |   7.491 |     216
украина         |                 1 |             168 |   6.611 |     223
род             |                 1 |               6 |   0.046 |     875
направляться    |                 1 |               6 |   0.046 |     876
выставлять      |                 1 